# Human-in-the-loop

There are certain tools that we don't trust a model to execute on its own. One thing we can do in such situations is require human approval before the tool is invoked.

## Setup

We'll need to install the following packages:

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

And set these environment variables:

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# If you'd like to use LangSmith, uncomment the below:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Chain

Suppose we have the following (dummy) tools and tool-calling chain:

In [11]:
from operator import itemgetter

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.runnables import Runnable
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


@tool
def count_emails(last_n_days: int) -> int:
    """Multiply two integers together."""
    return last_n_days * 2


@tool
def send_email(message: str, recipient: str) -> str:
    "Add two integers."
    return f"Successfully sent email to {recipient}."


tools = [count_emails, send_email]
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)


def tool_chain(function_api_output: dict) -> Runnable:
    tool_map = {tool.name: tool for tool in tools}
    chosen = tool_map[function_api_output["name"]]
    return itemgetter("arguments") | chosen


chain = model | JsonOutputFunctionsParser(args_only=False) | tool_chain
chain.invoke("how many emails did i get in the last 5 days?")

10

## Adding human approval

We can add a simple human approval step to our tool_chain function:

In [18]:
def tool_chain(function_api_output: dict) -> Runnable:
    msg = (
        f"Do you approve of the following input to the `{function_api_output['name']}` tool?\n\n{function_api_output['arguments']}\n\n"
        "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no."
    )
    resp = input(msg)
    approved = resp.lower() in ("yes", "y")
    if not approved:
        return f"Tool use not approved. Invoked tool {function_api_output['name']} with arguments {function_api_output['arguments']}"
    tool_map = {tool.name: tool for tool in tools}
    chosen = tool_map[function_api_output["name"]]
    return itemgetter("arguments") | chosen

In [19]:
chain = model | JsonOutputFunctionsParser(args_only=False) | tool_chain
chain.invoke("how many emails did i get in the last 5 days?")

Do you approve of the following input to the `count_emails` tool?

{'last_n_days': 5}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no. Y


10

In [20]:
chain.invoke("Send sally@gmail.com an email saying 'What's up homie'")

Do you approve of the following input to the `send_email` tool?

{'message': "What's up homie", 'recipient': 'sally@gmail.com'}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no. n


'Tool use not approved. Invoked tool send_email with arguments {\'message\': "What\'s up homie", \'recipient\': \'sally@gmail.com\'}'